In [ ]:
import sys
sys.path.append('..')  

import torch
import torch.nn as nn
from ml.models import SentimentModel

def test_model():
    print("Testing SentimentModel...")
    
    # Initialize model
    model = SentimentModel()
    print(f"Model initialized. Parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # Test data
    single_text = "This product is absolutely amazing!"
    batch_texts = [
        "Great quality, love it!",
        "Not worth the money, poor quality.",
        "It's okay, nothing special."
    ]
    
    print("\n=== FORWARD PASS TESTS ===")
    
    # Test 1: Single text forward pass
    print("\n1. Single text forward pass:")
    model.eval()
    with torch.no_grad():
        logits = model.forward(single_text)
        print(f"Input: '{single_text}'")
        print(f"Logits shape: {logits.shape}")
        print(f"Logits: {logits}")
        
    # Test 2: Batch forward pass
    print("\n2. Batch forward pass:")
    with torch.no_grad():
        batch_logits = model.forward(batch_texts)
        print(f"Batch size: {len(batch_texts)}")
        print(f"Logits shape: {batch_logits.shape}")
        print(f"Batch logits:\n{batch_logits}")
    
    # Test 3: Prediction methods
    print("\n3. Prediction methods:")
    predictions = model.predict(batch_texts)
    print(f"Probabilities:\n{predictions}")
    
    pred_with_conf = model.predict_with_confidence(batch_texts)
    print(f"Predictions: {pred_with_conf['predictions']}")
    print(f"Confidence scores: {pred_with_conf['confidence']}")
    
    print("\n=== BACKWARD PASS TEST ===")
    
    # Test 4: Backward pass (gradient computation)
    model.train()
    
    # Create dummy targets
    targets = torch.tensor([2, 0, 1])  # positive, negative, neutral
    
    # Forward pass
    logits = model.forward(batch_texts)
    
    # Compute loss
    criterion = nn.CrossEntropyLoss()
    loss = criterion(logits, targets)
    
    print(f"\n4. Backward pass:")
    print(f"Loss: {loss.item():.4f}")
    
    # Backward pass
    loss.backward()
    
    # Check if gradients were computed
    has_gradients = any(p.grad is not None for p in model.parameters())
    print(f"Gradients computed: {has_gradients}")
    
    # Print a few gradient norms to verify
    grad_norms = []
    for name, param in model.named_parameters():
        if param.grad is not None:
            grad_norm = param.grad.norm().item()
            grad_norms.append((name, grad_norm))
    
    print(f"\nSample gradient norms:")
    for name, norm in grad_norms[:5]:  # Show first 5
        print(f"  {name}: {norm:.6f}")
    
    print(f"\n✅ Model test completed successfully!")
    print(f"✅ Forward pass: Working")
    print(f"✅ Backward pass: Working") 
    print(f"✅ Ready for training!")

if __name__ == "__main__":
    test_model()


Testing SentimentModel...
Model initialized. Parameters: 66,365,187

=== FORWARD PASS TESTS ===

1. Single text forward pass:
Input: 'This product is absolutely amazing!'
Logits shape: torch.Size([1, 3])
Logits: tensor([[-0.2018,  0.2261,  0.3786]])

2. Batch forward pass:
Batch size: 3
Logits shape: torch.Size([3, 3])
Batch logits:
tensor([[-0.3371,  0.1203,  0.3568],
        [-0.4509,  0.1856,  0.3680],
        [-0.3447,  0.2293,  0.4077]])

3. Prediction methods:
Probabilities:
tensor([[0.2183, 0.3449, 0.4369],
        [0.1939, 0.3664, 0.4397],
        [0.2042, 0.3625, 0.4333]])
Predictions: [2 2 2]
Confidence scores: [0.43687063 0.4397237  0.43328735]

=== BACKWARD PASS TEST ===

4. Backward pass:
Loss: 1.1232
Gradients computed: True

Sample gradient norms:
  distilbert.embeddings.word_embeddings.weight: 0.436941
  distilbert.embeddings.position_embeddings.weight: 0.440324
  distilbert.embeddings.LayerNorm.weight: 0.026794
  distilbert.embeddings.LayerNorm.bias: 0.044529
  distilb

In [8]:
import sys
sys.path.append('..')  

from ml.inference import SentimentPredictor

predictor = SentimentPredictor(model_path="../ml/models/production_sentiment_model.pth")  # Uses new trained model

# Beauty product reviews (your training domain)
test_cases = [
    "This foundation is absolutely perfect for my skin tone!",
    "Worst mascara ever. Clumpy and flakes off within an hour.",
    "It's okay, nothing special but does the job I guess.",
    "Love love love this lipstick! The color is gorgeous and it lasts all day.",
    "Complete waste of money. Broke after one use.",
    "Pretty good quality for the price. Would recommend.",
    "Meh. Expected better based on the reviews.",
    "Holy grail product! Can't live without it now.",
    "Terrible customer service and the product arrived damaged.",
    "Works as expected. Nothing to complain about.",
    "Good color but terrible packaging",  # Mixed sentiment
    "I guess it's fine",  # Weak positive
    "Could be better",  # Ambiguous
    "Not bad",  # Double negative
    "Product product product"  # Nonsense
]

for text in test_cases:
    result = predictor.predict(text)
    print(f"Text: {text}")
    print(f"Prediction: {result['prediction']['label']} (confidence: {result['prediction']['confidence']:.3f})")
    print(f"Probabilities: neg={result['probabilities']['negative']:.3f}, neu={result['probabilities']['neutral']:.3f}, pos={result['probabilities']['positive']:.3f}")
    print("-" * 80)


Model loaded from ../ml/models/production_sentiment_model.pth
Text: This foundation is absolutely perfect for my skin tone!
Prediction: positive (confidence: 1.000)
Probabilities: neg=0.000, neu=0.000, pos=1.000
--------------------------------------------------------------------------------
Text: Worst mascara ever. Clumpy and flakes off within an hour.
Prediction: negative (confidence: 0.999)
Probabilities: neg=0.999, neu=0.001, pos=0.001
--------------------------------------------------------------------------------
Text: It's okay, nothing special but does the job I guess.
Prediction: neutral (confidence: 0.803)
Probabilities: neg=0.110, neu=0.803, pos=0.087
--------------------------------------------------------------------------------
Text: Love love love this lipstick! The color is gorgeous and it lasts all day.
Prediction: positive (confidence: 1.000)
Probabilities: neg=0.000, neu=0.000, pos=1.000
-------------------------------------------------------------------------------